In [11]:
from neuralop.models import FNO
from neuralop.training import AdamW
from neuralop.utils import count_model_params
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data import Dataset, Subset
import torch
import torch.nn as nn
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.interpolate import interp1d
from os.path import dirname, join as pjoin
import os
import io
import scipy.io as sio
import urllib
from neuralop import Trainer
from neuralop.losses.data_losses import LpLoss
import glob

# データダウンロード

In [12]:
def load_all_moments(root="../vlasov_random_data",max_cases=None):
    """
    root 以下の data_XXXX/moments.npz を全部読み込んで
    X: (Nsamples, 3, N), Y: (Nsamples, 1, N) を作る
    """
    X_list = []
    Y_list = []

    # data_0000, data_0001, ... を全部拾う
    folders = sorted(glob.glob(os.path.join(root, "data_*")))
    total = len(folders)

    if max_cases is not None:
        folders = folders[:max_cases]   # ← 指定数だけフォルダを使う

    print(f"Found {total} folders, loading {len(folders)} folders")


    for idx, folder in enumerate(folders):
        path = os.path.join(folder, "moments.npz")
        if not os.path.isfile(path):
            continue

        d = np.load(path)
        n = d["n"]
        u = d["u"]
        p = d["p"]
        dn_dx = d["dn_dx"]
        du_dx = d["du_dx"]
        dp_dx = d["dp_dx"]
        dq = d["dq_dx"]

        # すぐ torch tensor に変換
        X = torch.tensor(np.stack([n,u,p,dn_dx,du_dx,dp_dx],axis=1), dtype=torch.float32)
        Y = torch.tensor(dq[:,None,:], dtype=torch.float32)

        X_list.append(X)
        Y_list.append(Y)

        # プログレス表示
        if idx % 50 == 0:
            print(f"  loading... {idx}/{len(folders)}")

    # torch.cat で結合（高速 & メモリ節約）
    X = torch.cat(X_list, dim=0)
    Y = torch.cat(Y_list, dim=0)

    print("Done loading:", X.shape, Y.shape)
    return X, Y


# パラメータの設定

In [ ]:
# パラメータ
batch_size = 32 # バッチサイズ
num_epoch = 30 # エポック数
num_modes = 16 # フーリエ空間で使用するモードの数
num_channels = 64 # インプットとアウトプットの間の層の数
in_channels = 6 # インプット数
device = 'cuda' if torch.cuda.is_available() else 'cpu'
root="../vlasov_random_data"
print("Using device:", device)

2.10.0+cu128
Using device: cuda


# データセット作成

In [14]:
def compute_and_apply_scaler(X, Y, save_path="scaler.npz"):
    """
    X: (Nsamp, 3, N)
    Y: (Nsamp, 1, N)
    学習時の平均・標準偏差を計算し、保存＆正規化した X,Y を返す
    """
    # チャネルごとに flatten して mean/std
    mu_n = X[:, 0, :].mean()
    sig_n = X[:, 0, :].std()
    mu_u = X[:, 1, :].mean()
    sig_u = X[:, 1, :].std()
    mu_p = X[:, 2, :].mean()
    sig_p = X[:, 2, :].std()
    mu_dn = X[:, 3, :].mean()
    sig_dn = X[:, 3, :].std()
    mu_du = X[:, 4, :].mean()
    sig_du = X[:, 4, :].std()
    mu_dp = X[:, 5, :].mean()
    sig_dp = X[:, 5, :].std()

    mu_dq = Y[:, 0, :].mean()
    sig_dq = Y[:, 0, :].std()

    # 保存
    np.savez(
        save_path,
        mu_n=mu_n, sig_n=sig_n,
        mu_u=mu_u, sig_u=sig_u,
        mu_p=mu_p, sig_p=sig_p,
        mu_dn=mu_dn, sig_dn=sig_dn,
        mu_du=mu_du, sig_du=sig_du,
        mu_dp=mu_dp, sig_dp=sig_dp,
        mu_dq=mu_dq, sig_dq=sig_dq,
    )

    # 正規化
    X_norm = np.empty_like(X, dtype=np.float32)
    Y_norm = np.empty_like(Y, dtype=np.float32)

    X_norm[:, 0, :] = (X[:, 0, :] - mu_n) / sig_n
    X_norm[:, 1, :] = (X[:, 1, :] - mu_u) / sig_u
    X_norm[:, 2, :] = (X[:, 2, :] - mu_p) / sig_p
    X_norm[:, 3, :] = (X[:, 3, :] - mu_dn) / sig_dn
    X_norm[:, 4, :] = (X[:, 4, :] - mu_du) / sig_du
    X_norm[:, 5, :] = (X[:, 5, :] - mu_dp) / sig_dp

    Y_norm[:, 0, :] = (Y[:, 0, :] - mu_dq) / sig_dq

    return X_norm, Y_norm

In [15]:
class VlasovClosureDataset(Dataset):
    def __init__(self, X, Y):
        # X: (Nsamp, 6, N), Y: (Nsamp, 1, N) (float32想定)
        self.X = torch.from_numpy(X.astype(np.float32))
        self.Y = torch.from_numpy(Y.astype(np.float32))

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


# Loss Function

In [16]:
if 1:
    def l2loss(pred, **sample):
        criterion = torch.nn.MSELoss()
        return criterion(pred, sample["y"])

else:
    #
    l2loss = LpLoss(d=1, p=2, reduction="mean")

# モデル

In [17]:
def build_fno_model(n_modes=16, hidden_channels=64, n_layers=4, N_grid=64):
    """
    1D FNO モデルを構築
    """
    model = FNO(
        n_modes=(n_modes,),     # 1次元なので tuple で (n_modes,)
        hidden_channels=hidden_channels,
        in_channels=6,
        out_channels=1,
        n_layers=n_layers,
        max_n_modes=(N_grid,),  # グリッド数（通常は N）
    )
    return model

# 学習

データ読み込みと正規化

In [18]:
# ---------- データ読み込み ----------
X, Y = load_all_moments(root=root,max_cases=300)

# ---------- 正規化 ----------
X_norm, Y_norm = compute_and_apply_scaler(X, Y, save_path="scaler_random.npz")
print("After normalization: X", X_norm.shape, "Y", Y_norm.shape)

# ---------- Dataset & split ----------
full_dataset = VlasovClosureDataset(X_norm, Y_norm)

N_total = len(full_dataset)
N_train = int(0.8 * N_total)
N_val   = int(0.1 * N_total)
N_test  = N_total - N_train - N_val

train_ds, val_ds, test_ds = random_split(full_dataset, [N_train, N_val, N_test])
print(f"Dataset split: train={N_train}, val={N_val}, test={N_test}")

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)


Found 300 folders, loading 300 folders
  loading... 0/300
  loading... 50/300
  loading... 100/300
  loading... 150/300
  loading... 200/300
  loading... 250/300
Done loading: torch.Size([900000, 6, 64]) torch.Size([900000, 1, 64])
After normalization: X (900000, 6, 64) Y (900000, 1, 64)
Dataset split: train=720000, val=90000, test=90000


モデル, optimizer, loss_fnの定義

In [19]:
# ---------- モデル構築 ----------
N_grid = X.shape[-1]
model = build_fno_model(
    n_modes=16,
    hidden_channels=64,
    n_layers=4,
    N_grid=N_grid
).to(device)

print(model)

# ---------- オプティマイザ・損失 ----------
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

FNO(
  (positional_embedding): GridEmbeddingND()
  (fno_blocks): FNOBlocks(
    (convs): ModuleList(
      (0-3): 4 x SpectralConv(
        (weight): DenseTensor(shape=torch.Size([64, 64, 64]), rank=None)
      )
    )
    (fno_skips): ModuleList(
      (0-3): 4 x Flattened1dConv(
        (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      )
    )
    (channel_mlp): ModuleList(
      (0-3): 4 x ChannelMLP(
        (fcs): ModuleList(
          (0): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
          (1): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
        )
      )
    )
    (channel_mlp_skips): ModuleList(
      (0-3): 4 x SoftGating()
    )
  )
  (lifting): ChannelMLP(
    (fcs): ModuleList(
      (0): Conv1d(7, 128, kernel_size=(1,), stride=(1,))
      (1): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
    )
  )
  (projection): ChannelMLP(
    (fcs): ModuleList(
      (0): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (1): Conv1d(128, 1, kernel_size=

学習ループ

In [20]:
# ---------- 学習ループ ----------
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for xb, yb in train_loader:
        xb = xb.to(device)        # (B,3,N)
        yb = yb.to(device)        # (B,1,N)

        optimizer.zero_grad()
        y_pred = model(xb)        # (B,1,N) を想定
        loss = loss_fn(y_pred, yb)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * xb.size(0)

    train_loss /= len(train_loader.dataset)

    # ---------- 検証 ----------
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            y_pred = model(xb)
            loss = loss_fn(y_pred, yb)
            val_loss += loss.item() * xb.size(0)

    val_loss /= len(val_loader.dataset)

    print(f"[{epoch+1:02d}/{num_epochs}] train_loss={train_loss:.4e}, val_loss={val_loss:.4e}")
torch.save(model.state_dict(), "FNOmodel_from_vlasov_random.pth")

[01/10] train_loss=8.6597e-02, val_loss=6.8697e-02
[02/10] train_loss=6.3800e-02, val_loss=6.2030e-02
[03/10] train_loss=5.7051e-02, val_loss=5.5197e-02
[04/10] train_loss=5.3307e-02, val_loss=5.2749e-02
[05/10] train_loss=5.0848e-02, val_loss=5.0828e-02
[06/10] train_loss=4.9055e-02, val_loss=4.9632e-02
[07/10] train_loss=4.7635e-02, val_loss=4.8176e-02
[08/10] train_loss=4.6497e-02, val_loss=4.7077e-02
[09/10] train_loss=4.5599e-02, val_loss=4.6637e-02
[10/10] train_loss=4.4833e-02, val_loss=4.5748e-02
